In [1]:
import sys
# sys.path.append('/Users/bernardoloureiro/template-lib')

from utils.notebookhelpers.helpers import Helpers
from utils.dtos.templateOutputCollection import TemplateOutputCollection
from utils.dtos.variable import Metadata
from utils.dtos.templateOutput import TemplateOutput
from utils.dtos.templateOutput import OutputType
from utils.dtos.templateOutput import ChartType
import datetime
import logging
import pandas as pd
import numpy as np
import os
from dateutil import parser
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
import math
import scipy as scipy
import matplotlib.pyplot as plt
import seaborn as sns
import copy
%matplotlib inline
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import ADASYN
from sklearn.ensemble import RandomForestClassifier

from collections import Counter
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve, auc
# from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV

In [2]:
inputDatasetParameter=Helpers.get_or_create_input_dataset(
    name="inputBeneDataset2",
    metadata=Metadata(input_name='Bene_df', is_required=True, tooltip='Benefeciary Dataset for EDA'),
    local_context=locals()
)

# Gender = Helpers.get_or_create_input_var(
#     name="Gender",
#     metadata=Metadata(input_name="Longitude", is_required=True, tooltip="Longitude", multiple=False, datatypes=["DOUBLE", "LONG"], options=['FIELDS', 'CONSTANT'], datasets=['GeoDataset']),
#     local_context=locals()
# )

EDA_Beneficiary=Helpers.get_or_create_output_chart(
 name="EDA_Beneficiary_Data",
    metadata=Metadata(input_name='Gender_chart Chart Name', is_required=True, tooltip='Name of the chart to be created'),
    local_context=locals()
)

In [6]:
contextId = "EDA_Beneficiary_Data"
context = Helpers.getOrCreateContext(contextId = contextId, localVars=locals())

In [7]:
entities=Helpers.getAllEntities(context)
BeneDataset = inputDatasetParameter.value

EDA_Beneficiary_name=EDA_Beneficiary.value

KeyError: 'entityPaths'

In [ ]:
df = Helpers.getEntityData(context,BeneDataset)

In [ ]:
train_bene_df = df.copy()

In [2]:
pd.set_option('display.max_columns',30)
label_font_dict = {'family':'sans-serif','size':13.5,'color':'brown','style':'italic'}
title_font_dict = {'family':'sans-serif','size':16.5,'color':'Blue','style':'italic'}

In [ ]:
train_bene_df['Gender'] = train_bene_df['Gender'].apply(lambda val: 0 if val == 2 else 1)

In [ ]:
# Here, I'm displaying the distribution of BENEFICIARIES on the basis of GENDER 
with plt.style.context('seaborn'):
    plt.figure(figsize=(10,8))
    fig = train_bene_df['Gender'].value_counts().plot(kind='bar', color=['yellow','purple'])
    # Using the "patches" function we will get the location of the rectangle bars from the graph.
    ## Then by using those location(width & height) values we will add the annotations
    for p in fig.patches:
        width = p.get_width()
        height = p.get_height()
        x, y = p.get_xy()
        fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],2))+"%"}', (x + width/2, y + height*1.015), ha='center', fontsize=13.5)
    # Providing the labels and title to the graph
    plt.xlabel("Gender Code", fontdict=label_font_dict)
    plt.ylabel("Number or % share of patients\n", fontdict=label_font_dict)
    plt.grid(which='major', linestyle="--", color='lightgrey')
    plt.minorticks_on()
    plt.title("Distribution of BENEFICIARIES based on GENDER\n", fontdict=title_font_dict)

In [ ]:
imgChart = Helpers.createTemplateOutputPlotlibChart(context, chartTitle='Distribution of BENEFICIARIES based on GENDER',
                                                    plt=plt)

### **Q3. Lets calculate the AGE of every BENEFICIARY?**

In [4]:
train_bene_df['DOB'] = pd.to_datetime(train_bene_df['DOB'], format="%Y-%m-%d")

NameError: name 'train_bene_df' is not defined

In [ ]:
train_bene_df['Patient_Age_Year'] = train_bene_df['DOB'].dt.year
train_bene_df['Patient_Age_Month'] = train_bene_df['DOB'].dt.month

In [ ]:
bene_age_year_df = pd.DataFrame(train_bene_df['Patient_Age_Year'].value_counts()).reset_index(drop=False)
bene_age_year_df.columns= ['year','num_of_beneficiaries']
bene_age_year_df = bene_age_year_df.sort_values(by='year')

In [ ]:
# Here, I'm displaying the distribution of BENEFICIARIES on the basis of their YEAR of Birth 
with plt.style.context('seaborn'):
  plt.figure(figsize=(21,9))
  fig = sns.barplot(data=bene_age_year_df, x='year', y='num_of_beneficiaries', palette='inferno')
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],1))+"%"}', (x + width/2, y + height*1.025), ha='center', fontsize=13.5, rotation=90)
  # Providing the labels and title to the graph
  plt.xlabel("\nBeneficary YEAR of Birth", fontdict=label_font_dict)
  plt.xticks(rotation=90)
  plt.ylabel("Number or % share of patients\n", fontdict=label_font_dict)
  plt.minorticks_on()
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.title("Distribution of BENEFICIARIES based on their YEAR of birth\n", fontdict=title_font_dict)

In [ ]:
imgChart2 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES based on their YEAR of birth',
                                                    plt=plt)

In [ ]:
bene_age_month_df = pd.DataFrame(train_bene_df['Patient_Age_Month'].value_counts()).reset_index(drop=False)
bene_age_month_df.columns= ['month','num_of_beneficiaries']
bene_age_month_df = bene_age_month_df.sort_values(by='month')

In [ ]:
# Here, I'm displaying the distribution of BENEFICIARIES on the basis of their MONTH of Birth 
with plt.style.context('seaborn'):
  plt.figure(figsize=(12,8))
  fig = sns.barplot(data=bene_age_month_df, x='month', y='num_of_beneficiaries', palette='summer')
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],1))+"%"}', (x + width/2, y + height*1.025), ha='center', fontsize=13.5, rotation=90)
  # Providing the labels and title to the graph
  plt.xlabel("\nBeneficary MONTH of Birth\n", fontdict=label_font_dict)
  plt.xticks(rotation=90)
  plt.ylabel("Number or % share of patients", fontdict=label_font_dict)
  plt.minorticks_on()
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.title("Distribution of BENEFICIARIES based on their MONTH of birth\n", fontdict=title_font_dict)

In [ ]:
imgChart3 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES based on their MONTH of birth',
                                                    plt=plt)

In [ ]:
# 0 means ALIVE and 1 means DEAD
train_bene_df['Dead_or_Alive'] = train_bene_df['DOD'].apply(lambda val: 0 if val != val else 1)

In [ ]:
# Here, I'm displaying the distribution of whether BENEFICIARY is ALIVE or NOT?
with plt.style.context('seaborn'):
  plt.figure(figsize=(10,8))
  fig = train_bene_df['Dead_or_Alive'].value_counts().plot(kind='bar', color=['lightgreen','coral'])
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],2))+"%"}', (x + width/2, y + height*1.015), ha='center', fontsize=13.5)
  # Providing the labels and title to the graph
  plt.xlabel("Alive or Dead Status?", fontdict=label_font_dict)
  plt.xticks(labels=["ALIVE","DEAD"], ticks=[0,1], rotation=20)
  plt.ylabel("Number or % share of patients\n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.minorticks_on()
  plt.title("Distribution of BENEFICIARIES based on Alive or Dead Status\n", fontdict=title_font_dict)

In [ ]:
imgChart4 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES based on Alive or Dead Status',
                                                    plt=plt)

In [ ]:
train_bene_df['DOD'] = pd.to_datetime(train_bene_df['DOD'])

In [ ]:
# Greatest Date of Death in the TRAIN set for beneficiaries
max_bene_DOD = max(train_bene_df['DOD'].unique()[1:])
# For all NAN DODs filling the greatest Date of Death
train_bene_df['DOD'].fillna(value=max_bene_DOD, inplace=True)
train_bene_df['AGE'] = np.round(((train_bene_df['DOD'] - train_bene_df['DOB']).dt.days)/365.0,1)
train_bene_df.drop(labels=['DOD'],axis=1,inplace=True)

In [ ]:
# Here, I'm displaying the distribution of AGE of Beneficiaries?
with plt.style.context('seaborn'):
  plt.figure(figsize=(10,8))
  train_bene_df['AGE'].plot(kind='hist', color='purple')
  # Providing the labels and title to the graph
  plt.xlabel("\nBeneficiaries Age in years", fontdict=label_font_dict)
  plt.ylabel("Frequency of patients\n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.minorticks_on()
  plt.title("Distribution of BENEFICIARIES AGE", fontdict=title_font_dict)
  plt.legend();

In [ ]:
imgChart5 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES AGE',
                                                    plt=plt)

In [ ]:
def bene_age_brackets(val):
  if val >=1 and val <=40:
    return 'Young'
  elif val > 40 and val <=60:
    return 'Mid'
  elif val > 60 and val <= 80:
    return 'Old'
  else:
    return 'Very Old'

In [ ]:
train_bene_df['AGE_groups'] = train_bene_df['AGE'].apply(lambda age: bene_age_brackets(age))

In [ ]:
age_grps = list(train_bene_df['AGE_groups'].unique())
for grp in age_grps:
  # Here, I'm displaying the distribution of AGE GROUPS of Beneficiaries?
  with plt.style.context('seaborn'):
    plt.figure(figsize=(8,6))
    train_bene_df[train_bene_df['AGE_groups'] == grp]['AGE'].plot(kind='hist', color='grey')
    # Providing the labels and title to the graph
    plt.xlabel("\nBeneficiaries Age in years", fontdict=label_font_dict)
    plt.ylabel("Frequency of patients\n", fontdict=label_font_dict)
    plt.grid(which='major', linestyle="--", color='lightgrey')
    plt.minorticks_on()
    plt.title("Distribution of BENEFICIARIES Age group -- {}".format(str(grp).upper()), fontdict=title_font_dict)
    plt.legend();

In [ ]:

imgChart6 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES Age group',
                                                    plt=plt)

In [ ]:
# Here, I'm displaying the distribution of BENEFICIARIES on the basis of Human RACE
with plt.style.context('seaborn'):
  plt.figure(figsize=(10,8))
  fig = train_bene_df['Race'].value_counts().plot(kind='bar', color=['lightgreen','coral','purple','red'])
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],2))+"%"}', (x + width/2, y + height*1.02), ha='center', fontsize=13.5)
  # Providing the labels and title to the graph
  plt.xlabel("Human RACE", fontdict=label_font_dict)
  plt.xticks(labels=["Race_1","Race_2","Race_3","Race_5"], ticks=[0,1,2,3], rotation=10)
  plt.ylabel("Number or % share of patients\n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.minorticks_on()
  plt.title("Distribution of BENEFICIARIES on the basis of Human RACE\n", fontdict=title_font_dict)

In [ ]:
imgChart7 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES on the basis of Human RACE',
                                                    plt=plt)

In [ ]:
# Lets validate whether we have imbalance of males and females across the human races
with plt.style.context('seaborn'):
  plt.figure(figsize=(12,8))
  fig = train_bene_df.groupby(['Gender','Race'])['AGE'].count().plot(kind='bar', color=['lightgreen','coral','purple','red','lightgreen','coral','purple','red'])
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],2))+"%"}', (x + width/2, y + height*1.02), ha='center', fontsize=13.5)
  # Providing the labels and title to the graph
  plt.xlabel("Gender & Human RACE", fontdict=label_font_dict)
  plt.xticks(labels=[("Gender_0","Race_1"), ("Gender_0","Race_2"), ("Gender_0","Race_3"), ("Gender_0","Race_5"),("Gender_1","Race_1"), ("Gender_1","Race_2"), ("Gender_1","Race_3"), ("Gender_1","Race_5")],
             ticks=[0,1,2,3,4,5,6,7], rotation=80)
  plt.ylabel("Number or % share of patients\n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.minorticks_on()
  plt.title("Imbalance of males and females across the human races?\n", fontdict=title_font_dict)

In [ ]:
imgChart8 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Imbalance of males and females across the human races',
                                                    plt=plt)

In [ ]:
# Here, I'm displaying the distribution of BENEFICIARIES on the basis of "ChronicCond_KidneyDisease"
with plt.style.context('seaborn'):
  plt.figure(figsize=(10,8))
  fig = train_bene_df['ChronicCond_KidneyDisease'].value_counts().plot(kind='bar', color=['green','orange'])
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],2))+"%"}', (x + width/2, y + height*1.02), ha='center', fontsize=13.5, rotation=0)
  # Providing the labels and title to the graph
  plt.xlabel("\n Chronic Kidney Disease present or not?", fontdict=label_font_dict)
  plt.xticks(labels=["NO","YES"], ticks=[0,1], rotation=10, size=12)
  plt.ylabel("Number or % share of patients\n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.minorticks_on()
  plt.title("Distribution of BENEFICIARIES on the basis of 'ChronicCond_KidneyDisease'\n", fontdict=title_font_dict)

# I believe 2 means NO and 1 means YES
print(pd.DataFrame(train_bene_df['ChronicCond_KidneyDisease'].value_counts()),'\n')

In [ ]:
imgChart9 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES on the basis of ChronicCond_KidneyDisease',
                                                    plt=plt)

In [ ]:
# Lets validate whether we have beneficiaries with both RKD & CKD?
with plt.style.context('seaborn'):
  plt.figure(figsize=(12,8))
  fig = train_bene_df.groupby(['RenalDiseaseIndicator','ChronicCond_KidneyDisease'])['Gender'].count().plot(kind='bar', color=['orange','green','purple','red'])
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],2))+"%"}', (x + width/2, y + height*1.02), ha='center', fontsize=13.5)
  
  # Added the Description for one of the horizontal bar
  fig.annotate('This is interesting!!\n Beneficiaries with NO Long Term Kidney Malfunction but\n suffered from Kidney Failure.', 
               xy=(2.85, 0.85),  xycoords='data', xytext=(1.02, 0.65), textcoords='axes fraction', fontsize=12.5,
               arrowprops=dict(facecolor='black', shrink=0.03), horizontalalignment='right', verticalalignment='top')

  # Providing the labels and title to the graph
  plt.xlabel("\nRKD and CKD both are present?", fontdict=label_font_dict)
  plt.xticks(labels=[("RKD_No","CKD_Yes"), ("RKD_No","CKD_No"), ("RKD_Yes","CKD_Yes"), ("RKD_Yes","CKD_No")], ticks=[0,1,2,3], rotation=0, size=12)
  plt.ylabel("Number or % share of patients\n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.minorticks_on()
  plt.title("Imbalance b/w males and females across the human races?\n", fontdict=title_font_dict)

# RKD --> Renal Disease Indicator
# CKD --> Chronic Condition Kidney Disease
print(pd.DataFrame(train_bene_df.groupby(['RenalDiseaseIndicator','ChronicCond_KidneyDisease'])['Gender'].count()),"\n")

In [ ]:
imgChart10 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Imbalance b/w males and females across the human races?',
                                                    plt=plt)

# Updating charts

In [ ]:
# Here, I'm displaying the distribution of BENEFICIARIES on the basis of State
with plt.style.context('seaborn'):
  plt.figure(figsize=(20,9))
  fig = train_bene_df['State'].value_counts().plot(kind='bar')
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],2))+"%"}', (x + width/2, y + height*1.03), ha='center', fontsize=13.5, rotation=90)
  # Providing the labels and title to the graph
  plt.xlabel("\nState Codes", fontdict=label_font_dict)
  plt.ylabel("Number or % share of patients\n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.minorticks_on()
  plt.title("Distribution of BENEFICIARIES on the basis of States\n", fontdict=title_font_dict)

In [ ]:
imgChart11 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='"Distribution of BENEFICIARIES on the basis of States\n',
                                                    plt=plt)

In [ ]:
# Here, I'm displaying the distribution of BENEFICIARIES on the basis of NoOfMonths_PartACov
with plt.style.context('seaborn-poster'):
  plt.figure(figsize=(18,8))
  fig = train_bene_df['NoOfMonths_PartACov'].value_counts().plot(kind='bar', color='palegreen')
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],2))+"%"}', (x + width/2, y + height*1.03), ha='center', fontsize=13.5, rotation=90)
  # Providing the labels and title to the graph
  plt.xlabel("\nMonths for Part-A Coverage", fontdict=label_font_dict)
  plt.ylabel("Number or % share of patients\n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.minorticks_on()
  plt.title("Distribution of BENEFICIARIES on the basis of 'NoOfMonths_PartACov'", fontdict=title_font_dict)

In [ ]:
imgChart12 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle="Distribution of BENEFICIARIES on the basis of NoOfMonths_PartACov",
                                                    plt=plt)

In [ ]:
# Here, I'm displaying the distribution of BENEFICIARIES on the basis of NoOfMonths_PartBCov
with plt.style.context('seaborn-poster'):
  plt.figure(figsize=(18,8))
  fig = train_bene_df['NoOfMonths_PartBCov'].value_counts().plot(kind='bar', color='palegreen')
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],2))+"%"}', (x + width/2, y + height*1.03), ha='center', fontsize=13.5, rotation=90)
  # Providing the labels and title to the graph
  plt.xlabel("\nMonths for Part-B Coverage", fontdict=label_font_dict)
  plt.ylabel("Number or % share of patients\n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.minorticks_on()
  plt.title("Distribution of BENEFICIARIES on the basis of 'NoOfMonths_PartBCov'", fontdict=title_font_dict)

In [ ]:
imgChart13 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle="Distribution of BENEFICIARIES on the basis of 'NoOfMonths_PartBCov'",
                                                    plt=plt)

In [ ]:
# Here, I'm displaying the distribution of BENEFICIARIES on the basis of 'ChronicCond_Alzheimer'
with plt.style.context('seaborn-poster'):
  plt.figure(figsize=(12,8))
  fig = train_bene_df['ChronicCond_Alzheimer'].value_counts().plot(kind='bar', color=['palegreen','orange'])
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],2))+"%"}', (x + width/2, y + height*1.01), ha='center', fontsize=13.5, rotation=0)
  # Providing the labels and title to the graph
  plt.xlabel("\nHaving Chronic ALZH Disease?", fontdict=label_font_dict)
  plt.xticks(ticks=[0,1], labels=['NO', 'YES'], fontsize=13, rotation=30)
  plt.ylabel("Number or % share of patients\n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.minorticks_on()
  plt.title("Distribution of BENEFICIARIES on the basis of 'ChronicCond_Alzheimer'\n", fontdict=title_font_dict)

# 1 means +ve with Chronic ALZH Disease
# 2 means -ve with Chronic ALZH Disease
print(pd.DataFrame(train_bene_df['ChronicCond_Alzheimer'].value_counts()),"\n")

In [ ]:
imgChart14 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle="Distribution of BENEFICIARIES on the basis of 'ChronicCond_Alzheimer'\n",
                                                    plt=plt)

In [ ]:
# Here, I'm displaying the Total Annual Sum of Max IP Reimbursement for 'ChronicCond_Alzheimer'
with plt.style.context('seaborn-poster'):
  plt.figure(figsize=(12,8))
  fig = train_bene_df.groupby(['ChronicCond_Alzheimer'])['IPAnnualReimbursementAmt'].sum().plot(kind='bar', color=['orange','palegreen'])
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{str(round((height*100)/(train_bene_df["IPAnnualReimbursementAmt"].sum()),2))+"%"}', (x + width/2, y + height*1.01), ha='center', fontsize=13.5, rotation=0)
  # Providing the labels and title to the graph
  plt.xlabel("\nHaving Chronic ALZH Disease?", fontdict=label_font_dict)
  plt.xticks(ticks=[0,1], labels=['YES', 'NO'], fontsize=13, rotation=30)
  plt.ylabel("Total Annual Sum of Max IP Reimbursement \n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="--", color='lightgrey')
  plt.minorticks_on()
  plt.title("Total Annual Sum of Max IP Reimbursement : 'ChronicCond_Alzheimer'\n", fontdict=title_font_dict)

# 1 means +ve with Chronic ALZH Disease
# 2 means -ve with Chronic ALZH Disease
print(pd.DataFrame(train_bene_df.groupby(['ChronicCond_Alzheimer'])['IPAnnualReimbursementAmt'].sum()),"\n")

In [ ]:
imgChart16 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle="Total Annual Sum of Max IP Reimbursement : 'ChronicCond_Alzheimer'\n",
                                                    plt=plt)

In [ ]:
CC_ALZH_IP_R = pd.DataFrame(train_bene_df.groupby(['ChronicCond_Alzheimer'])['IPAnnualReimbursementAmt'].sum() / train_bene_df.groupby(['ChronicCond_Alzheimer'])['BeneID'].count())
CC_ALZH_IP_R.columns = ['AVG IP Reimbursement Amt']

In [ ]:
CC_ALZH_OP_R = pd.DataFrame(train_bene_df.groupby(['ChronicCond_Alzheimer'])['OPAnnualReimbursementAmt'].sum() / train_bene_df.groupby(['ChronicCond_Alzheimer'])['BeneID'].count())
CC_ALZH_OP_R.columns = ['AVG OP Reimbursement Amt']

In [ ]:
CC_ALZH_IP_D = pd.DataFrame(train_bene_df.groupby(['ChronicCond_Alzheimer'])['IPAnnualDeductibleAmt'].sum() / train_bene_df.groupby(['ChronicCond_Alzheimer'])['BeneID'].count())
CC_ALZH_IP_D.columns = ['AVG IP Co-payment Amt']

In [ ]:
CC_ALZH_OP_D = pd.DataFrame(train_bene_df.groupby(['ChronicCond_Alzheimer'])['OPAnnualDeductibleAmt'].sum() / train_bene_df.groupby(['ChronicCond_Alzheimer'])['BeneID'].count())
CC_ALZH_OP_D.columns = ['AVG OP Co-payment Amt']

In [ ]:
CC_ALZH_all_amts = pd.concat([CC_ALZH_IP_R, CC_ALZH_OP_R, CC_ALZH_IP_D, CC_ALZH_OP_D], axis=1)

In [ ]:
# Here, I'm displaying the Total Annual Sum of IP Co-payment for 'ChronicCond_Alzheimer'
with plt.style.context('seaborn-poster'):
  fig = CC_ALZH_all_amts.plot(kind='bar', colormap='rainbow')
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{round(height,0)}', (x + width/2, y + height*1.015), ha='center', fontsize=12, rotation=0)
  # Providing the labels and title to the graph
  plt.xlabel("\nHaving Chronic ALZH Disease?", fontdict=label_font_dict)
  plt.xticks(ticks=[0,1], labels=['YES', 'NO'], fontsize=13, rotation=30)
  plt.ylabel("Total Annual Sum \n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="-.", color='lightgrey')
  plt.minorticks_on()
  plt.title("Total Annual Sum of various amounts : 'ChronicCond_Alzheimer'\n", fontdict=title_font_dict)

# 1 means +ve with Chronic ALZH Disease
# 2 means -ve with Chronic ALZH Disease
print(CC_ALZH_all_amts,"\n")

In [ ]:
imgChart17 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle="Total Annual Sum of various amounts : 'ChronicCond_Alzheimer'\n",
                                                    plt=plt)

In [ ]:
CC_KD_IP_R = pd.DataFrame(train_bene_df.groupby(['ChronicCond_KidneyDisease'])['IPAnnualReimbursementAmt'].sum() / train_bene_df.groupby(['ChronicCond_KidneyDisease'])['BeneID'].count())
CC_KD_IP_R.columns = ['AVG IP Reimbursement Amt']
CC_KD_OP_R = pd.DataFrame(train_bene_df.groupby(['ChronicCond_KidneyDisease'])['OPAnnualReimbursementAmt'].sum() / train_bene_df.groupby(['ChronicCond_KidneyDisease'])['BeneID'].count())
CC_KD_OP_R.columns = ['AVG OP Reimbursement Amt']
CC_KD_IP_D = pd.DataFrame(train_bene_df.groupby(['ChronicCond_KidneyDisease'])['IPAnnualDeductibleAmt'].sum() / train_bene_df.groupby(['ChronicCond_KidneyDisease'])['BeneID'].count())
CC_KD_IP_D.columns = ['AVG IP Co-payment Amt']
CC_KD_OP_D = pd.DataFrame(train_bene_df.groupby(['ChronicCond_KidneyDisease'])['OPAnnualDeductibleAmt'].sum() / train_bene_df.groupby(['ChronicCond_KidneyDisease'])['BeneID'].count())
CC_KD_OP_D.columns = ['AVG OP Co-payment Amt']
CC_KD_all_amts = pd.concat([CC_KD_IP_R, CC_KD_OP_R, CC_KD_IP_D, CC_KD_OP_D], axis=1)
 

In [ ]:
# Here, I'm displaying the Total Annual Sum of IP Co-payment for 'ChronicCond_KidneyDisease'
with plt.style.context('seaborn-poster'):
  fig = CC_KD_all_amts.plot(kind='bar', colormap='rainbow')
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{round(height,0)}', (x + width/2, y + height*1.015), ha='center', fontsize=12, rotation=0)
  # Providing the labels and title to the graph
  plt.xlabel("\nHaving Chronic KD Disease?", fontdict=label_font_dict)
  plt.xticks(ticks=[0,1], labels=['YES', 'NO'], fontsize=13, rotation=30)
  plt.ylabel("Total Annual Sum \n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="-.", color='lightgrey')
  plt.minorticks_on()
  plt.title("Total Annual Sum of various amounts : 'ChronicCond_KidneyDisease'\n", fontdict=title_font_dict)

# 1 means +ve with Chronic KD Disease
# 2 means -ve with Chronic KD Disease
# print(CC_KD_all_amts,"\n")

In [ ]:
imgChart18 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle="Total Annual Sum of various amounts : 'ChronicCond_KidneyDisease'\n",
                                                    plt=plt)

In [ ]:
CC_CN_IP_R = pd.DataFrame(train_bene_df.groupby(['ChronicCond_Cancer'])['IPAnnualReimbursementAmt'].sum() / train_bene_df.groupby(['ChronicCond_Cancer'])['BeneID'].count())
CC_CN_IP_R.columns = ['AVG IP Reimbursement Amt']
CC_CN_OP_R = pd.DataFrame(train_bene_df.groupby(['ChronicCond_Cancer'])['OPAnnualReimbursementAmt'].sum() / train_bene_df.groupby(['ChronicCond_Cancer'])['BeneID'].count())
CC_CN_OP_R.columns = ['AVG OP Reimbursement Amt']
CC_CN_IP_D = pd.DataFrame(train_bene_df.groupby(['ChronicCond_Cancer'])['IPAnnualDeductibleAmt'].sum() / train_bene_df.groupby(['ChronicCond_Cancer'])['BeneID'].count())
CC_CN_IP_D.columns = ['AVG IP Co-payment Amt']
CC_CN_OP_D = pd.DataFrame(train_bene_df.groupby(['ChronicCond_Cancer'])['OPAnnualDeductibleAmt'].sum() / train_bene_df.groupby(['ChronicCond_Cancer'])['BeneID'].count())
CC_CN_OP_D.columns = ['AVG OP Co-payment Amt']
CC_CN_all_amts = pd.concat([CC_CN_IP_R, CC_CN_OP_R, CC_CN_IP_D, CC_CN_OP_D], axis=1)

In [ ]:
# Here, I'm displaying the Total Annual Sum of IP Co-payment for 'ChronicCond_Cancer'
with plt.style.context('seaborn-poster'):
  fig = CC_CN_all_amts.plot(kind='bar', colormap='rainbow')
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{round(height,0)}', (x + width/2, y + height*1.015), ha='center', fontsize=12, rotation=0)
  # Providing the labels and title to the graph
  plt.xlabel("\nHaving Chronic CN Disease?", fontdict=label_font_dict)
  plt.xticks(ticks=[0,1], labels=['YES', 'NO'], fontsize=13, rotation=30)
  plt.ylabel("Total Annual Sum \n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="-.", color='lightgrey')
  plt.minorticks_on()
  plt.title("Total Annual Sum of various amounts : 'ChronicCond_Cancer'\n", fontdict=title_font_dict)

# 1 means +ve with Chronic CN Disease
# 2 means -ve with Chronic CN Disease
# print(CC_CN_all_amts,"\n")

In [ ]:
imgChart19 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle="Total Annual Sum of various amounts : 'ChronicCond_Cancer'\n",
                                                    plt=plt)

In [ ]:
RKD_IP_R = pd.DataFrame(train_bene_df.groupby(['RenalDiseaseIndicator'])['IPAnnualReimbursementAmt'].sum() / train_bene_df.groupby(['RenalDiseaseIndicator'])['BeneID'].count())
RKD_IP_R.columns = ['AVG IP Reimbursement Amt']
RKD_OP_R = pd.DataFrame(train_bene_df.groupby(['RenalDiseaseIndicator'])['OPAnnualReimbursementAmt'].sum() / train_bene_df.groupby(['RenalDiseaseIndicator'])['BeneID'].count())
RKD_OP_R.columns = ['AVG OP Reimbursement Amt']
RKD_IP_D = pd.DataFrame(train_bene_df.groupby(['RenalDiseaseIndicator'])['IPAnnualDeductibleAmt'].sum() / train_bene_df.groupby(['RenalDiseaseIndicator'])['BeneID'].count())
RKD_IP_D.columns = ['AVG IP Co-payment Amt']
RKD_OP_D = pd.DataFrame(train_bene_df.groupby(['RenalDiseaseIndicator'])['OPAnnualDeductibleAmt'].sum() / train_bene_df.groupby(['RenalDiseaseIndicator'])['BeneID'].count())
RKD_OP_D.columns = ['AVG OP Co-payment Amt']
RKD_all_amts = pd.concat([RKD_IP_R, RKD_OP_R, RKD_IP_D, RKD_OP_D], axis=1)

In [ ]:
# Here, I'm displaying the Total Annual Sum of IP Co-payment for 'RenalDiseaseIndicator'
with plt.style.context('seaborn-poster'):
  fig = RKD_all_amts.plot(kind='bar', colormap='rainbow')
  # Using the "patches" function we will get the location of the rectangle bars from the graph.
  ## Then by using those location(width & height) values we will add the annotations
  for p in fig.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    fig.annotate(f'{round(height,0)}', (x + width/2, y + height*1.015), ha='center', fontsize=12, rotation=0)
  # Providing the labels and title to the graph
  plt.xlabel("\nHaving Chronic Renal Kidney Disease?", fontdict=label_font_dict)
  plt.xticks(ticks=[0,1], labels=['NO', 'YES'], fontsize=13, rotation=30)
  plt.ylabel("Total Annual Sum \n", fontdict=label_font_dict)
  plt.grid(which='major', linestyle="-.", color='lightgrey')
  plt.minorticks_on()
  plt.title("Total Annual Sum of various amounts : 'RenalDiseaseIndicator'\n", fontdict=title_font_dict)

# Y means +ve with Renal Kidney Disease
# 0 means -ve with Renal Kidney Disease

In [ ]:
imgChart19 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle="Total Annual Sum of various amounts : 'RenalDiseaseIndicator'\n",
                                                    plt=plt)

In [ ]:
def cal_display_percentiles(x_col, y_col, title_lbl, x_filter_code):
    """
    Description : This function is created for calculating and generating the percentiles for pre-disease indicators.
    
    Input: It accepts below parameters:
        1. x_col : Disease indicator feature name.
        2. y_col : Feature like re-imbursement or deductible amount whose percentiles you want to generate.
        3. title_lbl : Label to be provided in the title of the plot.
        4. x_filter_code : Category code for which you want to generate the percentiles.
        
    Output: It returns the dataframe having percentiles and their respective values for the specific disease indicator feature.
    And, it displays the pointplot graph of the same.
    """
    percentiles = []
    percentiles_vals = []

    # Calculating & storing the various percentiles and their respective values
    for val in [0.1,0.2,0.25,0.3,0.4,0.5,0.6,0.7,0.75,0.8,0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99,0.999,0.9999,0.99999,0.999999,1.0]:
        percentile = round(float(val*100),6)
        percentiles.append(percentile)

        percentile_val = round(train_bene_df[train_bene_df[x_col] == x_filter_code][y_col].quantile(val),1)
        percentiles_vals.append(percentile_val)

    # Creating the temp dataframe for displaying the results
    tmp_percentiles = pd.DataFrame([percentiles, percentiles_vals]).T
    tmp_percentiles.columns = ['Percentiles', 'Values']

    # Here, I'm displaying the Percentiles values for all disease code features
    with plt.style.context('seaborn-poster'):
        plt.figure(figsize=(15,7))
        sns.pointplot(data=tmp_percentiles, x='Percentiles', y='Values', markers="o", palette='spring')
        sns.pointplot(data=tmp_percentiles, x='Percentiles', y='Values', markers="", color='grey', linestyles="solid")
        # Providing the labels and title to the graph
        plt.xlabel("\nPercentiles", fontdict=label_font_dict)
        plt.xticks(rotation=90, size=12)
        plt.ylabel("Total Annual `{}` Sum \n".format(y_col), fontdict=label_font_dict)
        plt.grid(which='major', linestyle="-.", color='lightpink')
        plt.minorticks_on()
        plt.title("Percentile values of `{}` :: `{}`\n".format(y_col,title_lbl), fontdict=title_font_dict)
        
    return tmp_percentiles

In [ ]:
RKD_YES_IP_R_percentiles = cal_display_percentiles(x_col='RenalDiseaseIndicator', 
                                                   y_col='IPAnnualReimbursementAmt',
                                                   title_lbl="Renal Kidney Disease = YES",
                                                   x_filter_code='Y')



In [ ]:
imgChart20 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle="Renal Kidney Disease = YES",
                                                    plt=plt)

In [ ]:
RKD_NO_IP_R_percentiles = cal_display_percentiles(x_col='RenalDiseaseIndicator', 
                                                   y_col='IPAnnualReimbursementAmt',
                                                   title_lbl="Renal Kidney Disease = NO",
                                                   x_filter_code='0')

In [ ]:
imgChart21 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle="Renal Kidney Disease = NO",
                                                    plt=plt)

In [ ]:
with plt.style.context('seaborn-poster'):
    plt.figure(figsize=(12,12))
    sns.boxenplot(data=train_bene_df, x='AGE_groups', y='IPAnnualReimbursementAmt', hue='Gender', palette='cubehelix')
    plt.minorticks_on()
    plt.title("Spread of payment paid by payer across age groups\n", fontdict=title_font_dict)

In [ ]:
imgChart22 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle="Spread of IPAnnualReimbursementAmt  payment paid by payer across age groups\n",
                                                    plt=plt)

In [ ]:
with plt.style.context('seaborn-poster'):
    plt.figure(figsize=(12,12))
    sns.boxenplot(data=train_bene_df, x='AGE_groups', y='OPAnnualReimbursementAmt', hue='Gender', palette='cubehelix')
    plt.minorticks_on()
    plt.title("Spread of payment paid by payer across age groups\n", fontdict=title_font_dict)

In [ ]:
imgChart23 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle="Spread of OPAnnualReimbursementAmt  payment paid by payer across age groups\n",
                                                    plt=plt)

In [ ]:
with plt.style.context('seaborn-poster'):
    plt.figure(figsize=(12,8))
    sns.boxenplot(data=train_bene_df, x='Dead_or_Alive', y='IPAnnualReimbursementAmt', hue='Gender', palette='autumn')
    plt.minorticks_on()
    plt.title("Spread of payment paid by payer based on life status\n", fontdict=title_font_dict)

In [ ]:
imgChart24 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle="Spread of IPAnnualReimbursementAmt payment paid by payer based on life status\n",
                                                    plt=plt)

In [ ]:
with plt.style.context('seaborn-poster'):
    plt.figure(figsize=(12,8))
    sns.boxenplot(data=train_bene_df, x='Dead_or_Alive', y='OPAnnualReimbursementAmt', hue='Gender', palette='autumn')
    plt.minorticks_on()
    plt.title("Spread of payment paid by payer based on life status\n", fontdict=title_font_dict)

In [ ]:
imgChart25 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle="Spread of OPAnnualReimbursementAmt payment paid by payer based on life status\n",
                                                    plt=plt)

In [ ]:
# imgChart = Helpers.createTemplateOutputPlotlibChart(context, chartTitle='Distribution of BENEFICIARIES based on GENDER',
#                                                     plt=plt)
# imgChart2 = Helpers.createTemplateOutputPlotlibChart(context, 
#                                                     chartTitle='Distribution of BENEFICIARIES based on their YEAR of birth',
#                                                     plt=plt)
# imgChart3 = Helpers.createTemplateOutputPlotlibChart(context, 
#                                                     chartTitle='Distribution of BENEFICIARIES based on their MONTH of birth',
#                                                     plt=plt)
# imgChart4 = Helpers.createTemplateOutputPlotlibChart(context, 
#                                                     chartTitle='Distribution of BENEFICIARIES based on Alive or Dead Status',
#                                                     plt=plt)

# imgChart5 = Helpers.createTemplateOutputPlotlibChart(context, 
#                                                     chartTitle='Distribution of BENEFICIARIES AGE',
#                                                     plt=plt)
# imgChart6 = Helpers.createTemplateOutputPlotlibChart(context, 
#                                                     chartTitle='Distribution of BENEFICIARIES Age group',
#                                                     plt=plt)
# imgChart7 = Helpers.createTemplateOutputPlotlibChart(context, 
#                                                     chartTitle='Distribution of BENEFICIARIES on the basis of Human RACE',
#                                                     plt=plt)

# imgChart8 = Helpers.createTemplateOutputPlotlibChart(context, 
#                                                     chartTitle='Distribution of BENEFICIARIES on the basis of Human RACE',
#                                                     plt=plt)
# imgChart7 = Helpers.createTemplateOutputPlotlibChart(context, 
#                                                     chartTitle='Distribution of BENEFICIARIES on the basis of Human RACE',
#                                                     plt=plt)


outputCollection = Helpers.createOutputCollection(context)


outputCollection.addTemplateOutput(imgChart)
outputCollection.addTemplateOutput(imgChart2)
outputCollection.addTemplateOutput(imgChart3)
outputCollection.addTemplateOutput(imgChart4)
outputCollection.addTemplateOutput(imgChart5)
outputCollection.addTemplateOutput(imgChart6)
outputCollection.addTemplateOutput(imgChart7)
outputCollection.addTemplateOutput(imgChart8)
outputCollection.addTemplateOutput(imgChart9)
outputCollection.addTemplateOutput(imgChart10)

outputCollection.addTemplateOutput(imgChart11)
outputCollection.addTemplateOutput(imgChart12)
outputCollection.addTemplateOutput(imgChart13)
outputCollection.addTemplateOutput(imgChart14)
# outputCollection.addTemplateOutput(imgChart15)
outputCollection.addTemplateOutput(imgChart16)
outputCollection.addTemplateOutput(imgChart17)
outputCollection.addTemplateOutput(imgChart18)
outputCollection.addTemplateOutput(imgChart19)
outputCollection.addTemplateOutput(imgChart20)
outputCollection.addTemplateOutput(imgChart21)



Helpers.save(context)

Helpers.save(context)

outputCollection2 = Helpers.createOutputCollection(context)
imgChart2 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES based on their YEAR of birth',
                                                    plt=plt)

outputCollection2.addTemplateOutput(imgChart2)

Helpers.save(context)

outputCollection3 = Helpers.createOutputCollection(context)
imgChart3 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES based on their MONTH of birth',
                                                    plt=plt)

outputCollection3.addTemplateOutput(imgChart3)

Helpers.save(context)

outputCollection4 = Helpers.createOutputCollection(context)
imgChart4 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES based on Alive or Dead Status',
                                                    plt=plt)

outputCollection4.addTemplateOutput(imgChart4)

Helpers.save(context)

outputCollection5 = Helpers.createOutputCollection(context)
imgChart5 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES AGE',
                                                    plt=plt)

outputCollection5.addTemplateOutput(imgChart5)

Helpers.save(context)

outputCollection6 = Helpers.createOutputCollection(context)
imgChart6 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES Age group',
                                                    plt=plt)

outputCollection6.addTemplateOutput(imgChart6)

Helpers.save(context)

outputCollection7 = Helpers.createOutputCollection(context)
imgChart7 = Helpers.createTemplateOutputPlotlibChart(context, 
                                                    chartTitle='Distribution of BENEFICIARIES on the basis of Human RACE',
                                                    plt=plt)

outputCollection7.addTemplateOutput(imgChart7)

Helpers.save(context)

train_bene_df['Gender'] = train_bene_df['Gender'].apply(lambda val: 0 if val == 2 else 1)

# Here, I'm displaying the distribution of BENEFICIARIES on the basis of GENDER 
with plt.style.context('seaborn'):
    plt.figure(figsize=(10,8))
    fig = train_bene_df['Gender'].value_counts().plot(kind='bar', color=['yellow','purple'])
    # Using the "patches" function we will get the location of the rectangle bars from the graph.
    ## Then by using those location(width & height) values we will add the annotations
    for p in fig.patches:
        width = p.get_width()
        height = p.get_height()
        x, y = p.get_xy()
        fig.annotate(f'{str(round((height*100)/train_bene_df.shape[0],2))+"%"}', (x + width/2, y + height*1.015), ha='center', fontsize=13.5)
    # Providing the labels and title to the graph
    plt.xlabel("Gender Code", fontdict=label_font_dict)
    plt.ylabel("Number or % share of patients\n", fontdict=label_font_dict)
    plt.grid(which='major', linestyle="--", color='lightgrey')
    plt.minorticks_on()
    plt.title("Distribution of BENEFICIARIES based on GENDER\n", fontdict=title_font_dict)

outputCollection = Helpers.createOutputCollection(context)
imgChart = Helpers.createTemplateOutputPlotlibChart(context, chartTitle='Gender', plt=plt)

outputCollection.addTemplateOutput(imgChart)

Helpers.cre

Helpers.save(context)